# 02_graph_rag Playground

이 노트북은 **Graph RAG** 파이프라인을 실험하는 공간입니다.
Neo4j를 사용하여 감사 보고서의 구조적 정보(기관, 카테고리 등)를 그래프로 구축하고 검색합니다.

**사전 준비**:
1. `.env`에 Neo4j 접속 정보 설정 완료
2. `pip install -r 02_graph_rag/requirements.txt` 실행

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
from dotenv import load_dotenv

load_dotenv()

# Add root directory to sys.path to import config
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from pipeline_graph import GraphRAGPipeline
from ingest_graph import GraphIngestor
from config import Config

## 1. 데이터 적재 (Ingestion)
**주의**: 이 셀을 실행하면 기존 그래프 데이터가 초기화되고 다시 적재됩니다.

In [2]:
# ingestor = GraphIngestor()
# try:
#     data = ingestor.load_data(Config.DATA_PATH)
#     ingestor.clean_graph()
#     ingestor.create_constraints()
#     ingestor.ingest_data(data)
# except Exception as e:
#     print(f"Error: {e}")
# finally:
#     ingestor.close()

## 2. 파이프라인 초기화 및 테스트

In [3]:
pipeline = GraphRAGPipeline()
print("Graph Pipeline Initialized!")

Graph Pipeline Initialized!


### 2.1. 엔티티 추출 테스트
LLM이 질문에서 기관명과 카테고리를 잘 추출하는지 확인합니다.

In [4]:
queries = [
    "한국철도공사의 예산 관련 지적사항은?",
    "JDC의 인사 문제는?",
    "안전 사고 관련 사례 알려줘"
]

for q in queries:
    entities = pipeline.extract_entities(q)
    print(f"Q: {q} -> Entities: {entities}")

Extracting entities from: 한국철도공사의 예산 관련 지적사항은?
Q: 한국철도공사의 예산 관련 지적사항은? -> Entities: {'organization': '한국철도공사', 'category': '예산'}
Extracting entities from: JDC의 인사 문제는?
Q: JDC의 인사 문제는? -> Entities: {'organization': '제주국제자유도시개발센터', 'category': '인사'}
Extracting entities from: 안전 사고 관련 사례 알려줘
Q: 안전 사고 관련 사례 알려줘 -> Entities: {'organization': None, 'category': '안전'}


### 2.2. 전체 검색 및 답변 생성 테스트

In [9]:
query = "한국토지주택공사에서 근무관련 부적절 사례 알려줘"
answer = pipeline.run(query)

print("\n=== Answer ===\n")
print(answer)

Extracting entities from: 한국토지주택공사에서 근무관련 부적절 사례 알려줘
Search Targets - Org: 한국토지주택공사, Cat: 인사
Executing Cypher: MATCH (d:Document)-[:ISSUED_BY]->(o:Organization)-[:BELONGS_TO]->(c:Category) WHERE o.name CONTAINS $org AND c.name CONTAINS $cat RETURN d.title as title, d.problem_summary as problem, d.date as date LIMIT 5

=== Answer ===

죄송합니다. 조건에 맞는 감사 보고서를 찾지 못했습니다.
